In [1]:
import numpy as np

import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

import pickle as pk

BATCH_SIZE = 1
EPOCHS = 5
VALIDATION_SPLIT = 0.02
    
INPUT_DATA = "../data/problem_1_train.dat"
MODEL_NAME = "model_problem_1.h5"

2023-08-07 15:40:57.774380: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
labels = list()
sequences = list()
alphabet = set()

with open(INPUT_DATA, "rt") as inf:
    inf.readline() # kill the header
    for i, line in enumerate(inf):
        if len(line.strip()) == 0:
            continue
        line = line.split()
        labels.append(int(line[0]))
        sequences.append(list( int(x) for x in line[2:] ))
        alphabet = alphabet.union(set( int(x) for x in line[2:] ))
        if i % int(10e3) == 0:
            print("Line: ", i)
len(labels), len(sequences), alphabet

Line:  0
Line:  10000
Line:  20000
Line:  30000


(35621, 35621, {0, 1})

In [3]:
sequences[0], sequences[1], sequences[2], sequences[3], sequences[4]

([1, 0, 1, 1, 0, 1, 1, 1],
 [0, 0, 1, 0, 1, 0, 0],
 [1, 1, 0, 1, 0, 0],
 [1, 1, 1, 1, 0, 1],
 [0, 0, 1, 1, 1])

In [4]:
sequences = np.array(sequences)
labels = np.array(labels).reshape(-1, 1)
alphabet_map = {symbol: index for index, symbol in enumerate(alphabet)}

pk.dump(alphabet_map, open("alphabet_mapping.pk", "wb"))

sequences.shape, labels.shape, alphabet_map

/tmp/ipykernel_54942/3654861393.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences = np.array(sequences)


((35621,), (35621, 1), {0: 0, 1: 1})

In [5]:
sequences_one_hot = np.zeros((sequences.shape[0], sequences.shape[1], len(alphabet)))
for i, seq in enumerate(sequences):
    for j, sym in enumerate(seq):
        sequences_one_hot[i, j, alphabet_map[sym]] = 1
print("Before: ", sequences[:3])
print("Shape: ", sequences.shape)
print("After: ", sequences_one_hot[:3])
print("Shape: ", sequences_one_hot.shape)

IndexError: tuple index out of range

In [ ]:
def get_model(input_shape):
    OUTPUT_DIM = 1 # sigmoid output

    input_layer = Input(shape=input_shape)
    
    x = Bidirectional(LSTM(4))(input_layer)
    x_out = Dense(OUTPUT_DIM, activation="sigmoid")(x)

    model = Model(input_layer, x_out)

    model.compile(
        loss="binary_crossentropy",
        optimizer="adam",
        metrics=["binary_crossentropy"]
    )
    return model

In [ ]:
model = get_model(( sequences_one_hot.shape[1], sequences_one_hot.shape[2] ))

es = EarlyStopping(
    monitor="val_loss",
    patience=1,
    mode="auto")

model.fit(
    sequences_one_hot, 
    labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VALIDATION_SPLIT,
    callbacks=[es]
)

In [ ]:
model.save(MODEL_NAME)